<font size="5">**Cross validating the premodel**</font>
<br /><br />
<font size="3">All the timings and accuracy values are taken from the <font color = "green">database</font>. The average premodel feature extraction and prediction time were calculated <font color = "green">separately</font> and added to premodel inference time.</font>
 

<font size="3">First, we import the necessary libraries and start the *image generator* class.</font>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
import premodel
#!pip install --user imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
import image_generator
import database

In [2]:
image_generator.main()
imgGen = image_generator.ImageGenerator('images/val/images', 'images/val/val.txt', 'images/val/synset_words.txt')
label_list = imgGen.get_label_list()

images/val/images\ILSVRC2012_val_00016214.JPEG remote control, remote
images/val/images\ILSVRC2012_val_00044244.JPEG beacon, lighthouse, beacon light, pharos
images/val/images\ILSVRC2012_val_00042077.JPEG tape player
images/val/images\ILSVRC2012_val_00025035.JPEG robin, American robin, Turdus migratorius
images/val/images\ILSVRC2012_val_00027703.JPEG Shih-Tzu
images/val/images\ILSVRC2012_val_00040306.JPEG barrel, cask
images/val/images\ILSVRC2012_val_00020459.JPEG marimba, xylophone
images/val/images\ILSVRC2012_val_00045509.JPEG screw
images/val/images\ILSVRC2012_val_00026199.JPEG Blenheim spaniel
images/val/images\ILSVRC2012_val_00047886.JPEG plunger, plumber's helper


<font size="5">**Running the premodel**</font>
<br /><br />
<font size="3">We cross validate two premodels (<font color = "green">logistic regression</font> and <font color = "green">knn</font>) with <font color = "green">10-fold cross validation</font> on <font color = "green">20k</font> images. So, we obtain the <u>predicted models</u> for all <font color = "green">20k</font> images.</font>

In [3]:
preference = 'Top-1'                # here, we select our preference, Top-1 or Top-5
n = 1                          # 1 for top-1, 5 for top-5
pre_model = premodel.Premodel(preference)

In [4]:
output_log_reg = pre_model.prototype(200, ([('log_reg', 'log_reg', 'log_reg')]))  # logistic regression premodel

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Asus\miniconda3\envs\esrg\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Asus\miniconda3\envs\esrg\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "D:\DL-Inference\premodel.py", line 287, in CV_fold_worker
    X_train_second_level, X_val_second_level = undersample.fit_resample(X_train_second_level, X_val_second_level)
  File "C:\Users\Asus\miniconda3\envs\esrg\lib\site-packages\imblearn\base.py", line 83, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\Asus\miniconda3\envs\esrg\lib\site-packages\imblearn\under_sampling\_prototype_selection\_nearmiss.py", line 225, in _fit_resample
    X_class, n_neighbors=self.nn_.n_neighbors
  File "C:\Users\Asus\miniconda3\envs\esrg\lib\site-packages\sklearn\neighbors\_base.py", line 683, in kneighbors
    (n_samples_fit, n_neighbors)
ValueError: Expected n_neighbors <= n_s

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3

In [ ]:
output_knn = pre_model.prototype(200, ([('nn', 'nn', 'nn')]))      # knn premodel

In [ ]:
output_dt16 = pre_model.prototype(200, ([('dt16', 'dt16', 'dt16')]))      # knn premodel

<font size="5">**Testing the accuracy of the premodel**</font>
<br /><br />
<font size="3">First, we extract the predicted models for the 20k images for both <font color = "green">logistic regression</font> and <font color = "green">knn</font> premodels.</font>

In [ ]:
def img_num_from_name(img_name):          # obtaining the image number from image name
    ext = []
    for i in range(len(img_name)):
        if i > 14 and i < 23:
            ext.append(img_name[i])
    j = 0
    for i in range(len(ext)):
        if ext[i] == '0':
            j = j + 1
        if ext[i] != '0':
            break
    rightNum = []
    for i in range(j, len(ext)):
        rightNum.append(ext[i])
    rightNum = ''.join(rightNum)
    rightNum = int(rightNum)
    return rightNum


predicted_models_log_reg = []                              # models predicted by logistic regression premodel
for i in range(10):
    for j in range(len(output_log_reg[5][i][1][0])):
        img_paths = (output_log_reg[5][i][1][0][j][0])
        img_nums = (img_num_from_name(img_paths))
        if output_log_reg[5][i][1][0][j][4] == 'tf-mobilenet_v1' and output_log_reg[5][i][1][0][j][3] == 1:
            predicted_models_log_reg.append([img_nums, 'mobilenet_v1'])
        elif output_log_reg[5][i][1][0][j][4] == 'tf-inception_v4' and output_log_reg[5][i][1][0][j][3] == 2:
            predicted_models_log_reg.append([img_nums, 'inception_v4'])
        elif output_log_reg[5][i][1][0][j][4] == 'tf-resnet_v1_152' and output_log_reg[5][i][1][0][j][3] == 3:
            predicted_models_log_reg.append([img_nums, 'resnet_v1_152'])
        elif output_log_reg[5][i][1][0][j][4] == 'failed':
            predicted_models_log_reg.append([img_nums, 'failed'])
        

predicted_models_knn = []                                    # models predicted by knn premodel            
for i in range(10):
    for j in range(len(output_knn[5][i][1][0])):
        img_paths = (output_knn[5][i][1][0][j][0])
        img_nums = (img_num_from_name(img_paths))
        if output_knn[5][i][1][0][j][4] == 'tf-mobilenet_v1' and output_knn[5][i][1][0][j][3] == 1:
            predicted_models_knn.append([img_nums, 'mobilenet_v1'])
        elif output_knn[5][i][1][0][j][4] == 'tf-inception_v4' and output_knn[5][i][1][0][j][3] == 2:
            predicted_models_knn.append([img_nums, 'inception_v4'])
        elif output_knn[5][i][1][0][j][4] == 'tf-resnet_v1_152' and output_knn[5][i][1][0][j][3] == 3:
            predicted_models_knn.append([img_nums, 'resnet_v1_152'])
        elif output_knn[5][i][1][0][j][4] == 'failed':
            predicted_models_knn.append([img_nums, 'failed'])
            

predicted_models_dt16 = []                                    # models predicted by dt16 premodel            
for i in range(10):
    for j in range(len(output_dt16[5][i][1][0])):
        img_paths = (output_dt16[5][i][1][0][j][0])
        img_nums = (img_num_from_name(img_paths))
        if output_dt16[5][i][1][0][j][4] == 'tf-mobilenet_v1' and output_dt16[5][i][1][0][j][3] == 1:
            predicted_models_dt16.append([img_nums, 'mobilenet_v1'])
        elif output_dt16[5][i][1][0][j][4] == 'tf-inception_v4' and output_dt16[5][i][1][0][j][3] == 2:
            predicted_models_dt16.append([img_nums, 'inception_v4'])
        elif output_dt16[5][i][1][0][j][4] == 'tf-resnet_v1_152' and output_dt16[5][i][1][0][j][3] == 3:
            predicted_models_dt16.append([img_nums, 'resnet_v1_152'])
        elif output_dt16[5][i][1][0][j][4] == 'failed':
            predicted_models_dt16.append([img_nums, 'failed'])

In [ ]:
print(len(predicted_models_log_reg))

In [ ]:
print(len(predicted_models_knn))

In [ ]:
print(len(predicted_models_dt16))

<font size="3">Now, we obtain the <font color = "green">accuracy</font> and <font color = "green">inference time</font> results of the predicted models from the database.</font>

In [ ]:
accuracy_premodel = 0
time_premodel = 0
times = 0
log_reg_premodel_overhead = 42.39268                                      # average logistic regression premodel overhead

for i in range(len(predicted_models_log_reg)):
    if predicted_models_log_reg[i][1] != 'failed':
        times = times + 1                          # get_model_top_n (next line), where n is 1 or 5
        accuracy_premodel = accuracy_premodel + (database.get_model_top_n("inference", predicted_models_log_reg[i][0], predicted_models_log_reg[i][1], n))
        time_premodel = time_premodel + (database.get_model_time("inference", predicted_models_log_reg[i][0], predicted_models_log_reg[i][1]))

log_reg_accuracy = accuracy_premodel/len(predicted_models_log_reg)               # logistic regression premodel accuracy
log_reg_time = (time_premodel/times) + log_reg_premodel_overhead                 # logistic regression premodel inference time       
print("Logistic regression premodel accuracy = {}".format(log_reg_accuracy))
print("Logistic regression premodel average inference time = {}".format(log_reg_time))

In [ ]:
accuracy_premodel = 0
time_premodel = 0
times = 0
knn_premodel_overhead = 43.00882                                           # average knn premodel overhead

for i in range(len(predicted_models_knn)):
    if predicted_models_knn[i][1] != 'failed':
        times = times + 1                           # get_model_top_n (next line), where n is 1 or 5
        accuracy_premodel = accuracy_premodel + (database.get_model_top_n("inference", predicted_models_knn[i][0], predicted_models_knn[i][1], n))
        time_premodel = time_premodel + (database.get_model_time("inference", predicted_models_knn[i][0], predicted_models_knn[i][1]))

knn_accuracy = accuracy_premodel/len(predicted_models_knn)                        # knn premodel accuracy
knn_time = (time_premodel/times) + knn_premodel_overhead                             # knnn premodel inference time
print("KNN premodel accuracy = {}".format(knn_accuracy))
print("KNN premodel average inference time = {}".format(knn_time))

In [ ]:
accuracy_premodel = 0
time_premodel = 0
times = 0
dt16_premodel_overhead = 43.00882                                           # average dt16 premodel overhead

for i in range(len(predicted_models_dt16)):
    if predicted_models_dt16[i][1] != 'failed':
        times = times + 1                           # get_model_top_n (next line), where n is 1 or 5
        accuracy_premodel = accuracy_premodel + (database.get_model_top_n("inference", predicted_models_dt16[i][0], predicted_models_dt16[i][1], n))
        time_premodel = time_premodel + (database.get_model_time("inference", predicted_models_dt16[i][0], predicted_models_dt16[i][1]))

dt16_accuracy = accuracy_premodel/len(predicted_models_dt16)                        # dt16 premodel accuracy
dt16_time = (time_premodel/times) + dt16_premodel_overhead                             # dt16 premodel inference time
print("DT16 premodel accuracy = {}".format(dt16_accuracy))
print("DT16 premodel average inference time = {}".format(dt16_time))

<font size="5">**Comparing the results to MobileNet, Inception and ResNet**</font>
<br /><br />
<font size="3">We start by getting the <font color = "green">image numbers</font> of the 20k images.</font>

In [ ]:
img_nums = []
for i in range(1, 201):                        # the same range of images is selected
    img_nums.append(i)

<font size="3">Finally, we obtain the <font color = "green">accuracy</font> and <font color = "green">inference time</font> results for each DNN separately from the database.</font>

In [ ]:
mobilenet_right = 0
mobilenet_time = 0
for i in range(len(img_nums)):                            # obtaining accuracy and inference time for each image
    mobilenet_right = mobilenet_right + (database.get_model_top_n("inference", img_nums[i], 'mobilenet_v1', n))
    mobilenet_time = mobilenet_time + (database.get_model_time("inference", img_nums[i], 'mobilenet_v1'))
print("Mobilenet top-{} accuracy = {}".format(n, mobilenet_right/len(img_nums)))          # accuracy over 20k images
print("Mobilenet average inference time = {}".format(mobilenet_time/len(img_nums)))   # average inference time over 20k images

inception_right = 0
inception_time = 0
for i in range(len(img_nums)):
    inception_right = inception_right + (database.get_model_top_n("inference", img_nums[i], 'inception_v4', n))
    inception_time = inception_time + (database.get_model_time("inference", img_nums[i], 'inception_v4'))
print("Inception top-{} accuracy = {}".format(n, inception_right/len(img_nums)))
print("Inception average inference time = {}".format(inception_time/len(img_nums)))

resnet_right = 0
resnet_time = 0
for i in range(len(img_nums)):
    resnet_right = resnet_right + (database.get_model_top_n("inference", img_nums[i], 'resnet_v1_152', n))
    resnet_time = resnet_time + (database.get_model_time("inference", img_nums[i], 'resnet_v1_152'))
print("ResNet top-{} accuracy = {}".format(n, resnet_right/len(img_nums)))
print("ResNet average inference time = {}".format(resnet_time/len(img_nums)))

<font size="3">Here, we plot the results for comparison.</font>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
model_names = ['Mobilenet', 'Inception', 'Resnet', 'Log reg prem', 'KNN prem']
accuracy = [mobilenet_right/len(img_nums), inception_right/len(img_nums),
            resnet_right/len(img_nums), log_reg_accuracy, knn_accuracy, dt16_accuracy]

for i in range(len(accuracy)):
    accuracy[i] = accuracy[i]*100

ypos = np.arange(len(model_names))

plt.xticks(ypos, model_names)
plt.ylabel("accuracy (%)")
plt.title("{} accuracy".format(preference))
plt.bar(ypos, accuracy)
plt.ylim(top = 85)
plt.ylim(bottom = 65)
plt.legend()

In [ ]:
times = [mobilenet_time/len(img_nums), inception_time/len(img_nums),
         resnet_time/len(img_nums), log_reg_time, knn_time, dt16_time]
times_premodel = [0, 0, 0, log_reg_premodel_overhead, knn_premodel_overhead, dt16_premodel_overhead]

for i in range(len(times)):
    times[i] = times[i]/1000

for i in range(len(times_premodel)):
    times_premodel[i] = times_premodel[i]/1000
    
plt.xticks(ypos, model_names)
plt.ylabel("inference time (s)")
plt.title("Inference time")
plt.bar(ypos, times, label = "Inference")
plt.bar(ypos, times_premodel, label = "Premodel")
plt.legend()